In [1]:
import csv
with open('/home/1739678401/2024.9dataclean/MMGNN_0901/data/1-s2.0-S1385894721008925-mmc1.txt', 'r') as txt_file:
    lines = txt_file.readlines()

csv_data = []
for line in lines:
    parts = line.strip().split(',')
    csv_data.append(parts)

with open('/home/1739678401/2024.9dataclean/MMGNN_0901/data/output.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(csv_data)

In [ ]:
import pandas as pd

data = pd.read_csv('/home/1739678401/2024.9dataclean/MMGNN_0901/data/output.csv')
all_smiles = data.iloc[:, 0].tolist() + data.iloc[:, 1].tolist()
unique_smiles = list(set(all_smiles))

print(len(unique_smiles))

In [4]:
new_data = pd.DataFrame({'SMILES': unique_smiles})
new_data.to_csv('unique_smiles.csv', index=False)

In [1]:
import numpy as np
import hnswlib
from numpy import linalg as LA

def cosine_sim(x,y):
    return np.dot(x,y)/LA.norm(x)/LA.norm(y)

class mol:
    def __init__(self,smiles0,smiles1,feat):
        self.smiles0=smiles0
        self.smiles1=smiles1
        self.feat=feat

class infogain:
    def __init__(self, initial_points=None, n=3000000, keep_seed=False, submodular_k=4):
        self.clusters = None
        self.target_n = n
        self.current_n = 0
        self.keep_seed = keep_seed
        self.dim = 128*2
        self.submodular_k = submodular_k

        if initial_points: 
            self.data = initial_points.copy()
            self.current_n = len(initial_points)
            if keep_seed:
                self.seeded_num = len(initial_points)
            self.dim = len(initial_points[0].feat)
        else:
            self.data = []
        self.submodular_gain = [(1)]*len(self.data)
        # initialize HNSW index
        self.knn_graph = hnswlib.Index(space='cosine', dim=self.dim)
        self.knn_graph.init_index(max_elements=n, ef_construction=100, M=48, allow_replace_deleted = False)
        self.precluster(initial_points)
        self.knn_graph.set_ef(32)

    def precluster(self, initial_points):
    # Starting from some initial points (the cleaner the better) to do online selection
        if initial_points is None or initial_points==[]: return
        for idx,data in enumerate(self.data):
            data.index = idx

        for idx,data in enumerate(self.data):
            self.submodular_gain[idx] = self.submodular_func(data, True)
            self.knn_graph.add_items(data.feat, idx)

    def submodular_func(self, data, skip_one=False):
        if self.knn_graph.get_current_count()==0:
            return (1.)
        k = min(self.knn_graph.get_current_count(), self.submodular_k)
        near_label,near_distances = self.knn_graph.knn_query(data.feat, k)
        return np.mean(near_distances)

    def add_item(self, data):
        data.index = self.current_n
        self.data.append(data)
        self.knn_graph.add_items(data.feat, self.current_n)
        self.current_n+=1

    def replace_item(self, data, index):
        # Not used in current work but provide for future extension on replacing samples
        data_to_rep = self.data[index]
        n_index = data_to_rep.index
        data.index = self.current_n
        self.knn_graph.mark_deleted(n_index)
        self.knn_graph.add_items(data.I_feat, self.current_n, replace_deleted = True)
        self.data[index] = data
        self.current_n+=1

    def process_item(self, data,):
        # find near clusters
        # go into nearest clusters to search near neighbour
        # calculate corresponding threshold to decide if try to add or not
        gain = self.submodular_func(data)
        self.add_item(data)
        self.submodular_gain.append(gain)

    def final_gains(self):
        return self.submodular_gain


In [3]:
import torch

dic = torch.load('/home/1739678401/2024.9dataclean/MMGNN_0901/temp_data/uq_smiles.pt')

import pandas as pd
def file_to_mol(filepath):
    df = pd.read_csv(filepath, usecols=[0, 1])
    res = []
    for i, row in df.iterrows():
        tensor1, _ = dic[row[0]]
        tensor2, _ = dic[row[1]]
        feat = torch.cat((tensor1, tensor2), dim=0).to(dtype=torch.float32).cpu().detach().numpy().reshape(256)
        #print(feat.shape)
        res.append(mol(row[0],row[1],feat))
    return res

filepath='/home/1739678401/2024.9dataclean/MMGNN_0901/temp_data/1M.csv'
res=file_to_mol(filepath)

In [4]:
IG=infogain(res)
mn_finalgains=IG.final_gains()
print(len(mn_finalgains))
import numpy as np
gains = np.array([ (gain) for gain in mn_finalgains])

1000000


In [11]:
thr = np.quantile(gains,0.99)
gains=gains/thr
gains[gains>1]=1
gains[gains<0]=0

In [2]:
import pandas as pd
data = pd.read_csv('/home/1739678401/2024.9dataclean/MMGNN_0901/temp_data/1M.csv')

#selected_mask = np.random.rand(len(data)) < gains-0.05
selected_data = data[::100]
print(len(selected_data))
selected_data.to_csv('/home/1739678401/2024.9dataclean/MMGNN_0901/temp_data/theory_random.csv')

10000


In [3]:
import csv

input_file = '/home/1739678401/2024.9dataclean/MMGNN_0901/temp_data/theory_random.csv'
output_file = '/home/1739678401/2024.9dataclean/MMGNN_0901/data/theory_random.csv'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    for row in reader:
        new_row = row[1:4]  # 提取第二到第四列的数据
        writer.writerow(new_row)